<a href="https://colab.research.google.com/github/deepak23232323/master-project/blob/main/20__Train_%26_80__Test_Samples_Bagging_%26_Boosting_Classifiers_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as empatr_pnds
import numpy as empatr_nup
from sklearn.model_selection import train_test_split as empatrtrts
from sklearn.metrics import roc_auc_score as empatrtrc

In [ ]:
EmpAtri = empatr_pnds.read_csv('preprocessed attrition.csv')
EmpAtri

,Unnamed: 0,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,41,1,2,1102,2,1,2,1,1,...,3,1,0,8,0,1,6,4,0,5
1,1,49,0,1,279,1,8,1,1,2,...,4,4,1,10,3,3,10,7,1,7
2,2,37,1,2,1373,1,2,2,4,4,...,3,2,0,7,3,3,0,0,0,0
3,3,33,0,1,1392,1,3,4,1,5,...,3,3,0,8,3,3,8,7,3,0
4,4,27,0,2,591,1,2,1,3,7,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,1465,36,0,1,884,1,23,2,3,2061,...,3,3,1,17,3,3,5,2,0,3
1466,1466,39,0,2,613,1,6,1,3,2062,...,3,1,1,9,5,3,7,7,1,7
1467,1467,27,0,2,155,1,4,3,1,2064,...,4,2,1,6,0,3,6,2,0,3
1468,1468,49,0,1,1023,2,2,3,3,2065,...,3,4,0,17,3,2,9,6,0,8


In [ ]:
EmpAtri['Attrition'].value_counts()

,count
Attrition,
0,1233
1,237


In [ ]:
## Input and Output ##
EmPAtritX = EmpAtri.drop('Attrition',axis=1)
EmPAtritY = EmpAtri['Attrition']

In [ ]:
## Splitting into training 20%  & testing 80% ###
EmPAtritX_t, EmPAtritX_ts, EmPAtritY_t, EmPAtritY_ts = empatrtrts(EmPAtritX, EmPAtritY, test_size=0.8, random_state=6)

print("20% Train Shape : ",EmPAtritX_t.shape)
print("80% Test Shape  : ",EmPAtritX_ts.shape)


20% Train Shape :  (294, 32)
80% Test Shape  :  (1176, 32)


In [ ]:
import warnings as empatrwrn
empatrwrn.filterwarnings('ignore')

#Bagging Classifier

In [ ]:
class Basic_ML_EmPAtrit:
    def fit_EmPAtrit(self, EmPAtritX_t, EmPAtritY_t):
        self.thr = empatr_nup.mean(EmPAtritX_t, axis=0)

    def pr_EmPAtrit(self, EmPAtritX_t):

        return (EmPAtritX_t >= self.thr).astype(int).sum(axis=1) >= (EmPAtritX_t.shape[1] / 2)

    def pr_pro_EmPAtrit(self, EmPAtritX_t):
        prob= empatr_nup.mean(EmPAtritX_t >= self.thr, axis=1)
        return prob

class Bagg_EmPAtrit:
    def __init__(self, base_est, n_est=10):
        self.base_est = base_est
        self.n_est = n_est
        self.ML_EmPAtrit_s = []

    def fit_EmPAtrit(self, EmPAtritX_t, EmPAtritY_t):
        for _ in range(self.n_est):
            indices = empatr_nup.random.choice(len(EmPAtritY_t), len(EmPAtritY_t), replace=True)
            ML_EmPAtrit = self.base_est()
            ML_EmPAtrit.fit_EmPAtrit(EmPAtritX_t.iloc[indices].values, EmPAtritY_t.iloc[indices].values)
            self.ML_EmPAtrit_s.append(ML_EmPAtrit)

    def pr_EmPAtrit(self, EmPAtritX_t):
        EmPAtrit_PP = empatr_nup.zeros((EmPAtritX_t.shape[0], self.n_est))
        for ii, md in enumerate(self.ML_EmPAtrit_s):
            EmPAtrit_PP[:, ii] = md.pr_EmPAtrit(EmPAtritX_t.values)
        return empatr_nup.round(empatr_nup.mean(EmPAtrit_PP, axis=1)).astype(int)

    def pr_pro_EmPAtrit(self, EmPAtritX_ts):

        probs = empatr_nup.zeros((EmPAtritX_ts.shape[0], self.n_est))
        for ii, md in enumerate(self.ML_EmPAtrit_s):
            probs[:, ii] = md.pr_pro_EmPAtrit(EmPAtritX_ts.values)
        return empatr_nup.mean(probs, axis=1)

def ev_EmPAtrit(EmPAtritX_ts, EmPAtritP_ts):
    EmPAtrit_A = empatr_nup.mean(EmPAtritX_ts == EmPAtritP_ts)
    EmPAtrit_B = empatr_nup.sum((EmPAtritX_ts == 1) & (EmPAtritP_ts == 1)) / empatr_nup.sum(EmPAtritP_ts == 1) if empatr_nup.sum(EmPAtritP_ts == 1) > 0 else 0
    EmPAtrit_C = empatr_nup.sum((EmPAtritX_ts == 1) & (EmPAtritP_ts == 1)) / empatr_nup.sum(EmPAtritX_ts == 1) if empatr_nup.sum(EmPAtritX_ts == 1) > 0 else 0
    EmPAtrit_D = (2 * EmPAtrit_B * EmPAtrit_C) / (EmPAtrit_B + EmPAtrit_C) if (EmPAtrit_B + EmPAtrit_C) > 0 else 0
    return EmPAtrit_A, EmPAtrit_B, EmPAtrit_C, EmPAtrit_D

EmPAtrit_Scratch_Mod = Bagg_EmPAtrit(Basic_ML_EmPAtrit, n_est=10)
EmPAtrit_Scratch_Mod.fit_EmPAtrit(EmPAtritX_t, EmPAtritY_t)
EmPAtritP_ts = EmPAtrit_Scratch_Mod.pr_EmPAtrit(EmPAtritX_ts)

# Evaluate
EmPAtrit_A, EmPAtrit_B, EmPAtrit_C, EmPAtrit_D = ev_EmPAtrit(EmPAtritY_ts, EmPAtritP_ts)
print(f"Accuracy: {EmPAtrit_A:.2f}, Precision: {EmPAtrit_B:.2f}, Recall: {EmPAtrit_C:.2f}, F1-Score: {EmPAtrit_D:.2f}")

EmPAtritP_ts = EmPAtrit_Scratch_Mod.pr_pro_EmPAtrit(EmPAtritX_ts)
auc_roc_EmPAtrit = empatrtrc(EmPAtritY_ts, EmPAtritP_ts)
print(f"AUC-ROC: {auc_roc_EmPAtrit:.2f}")

Accuracy: 0.51, Precision: 0.11, Recall: 0.30, F1-Score: 0.16
AUC-ROC: 0.39
